In [ ]:
import pygame
import sys
import math

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600  # Screen dimensions
FPS = 60  # Frames per second
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

# Physics constants
GRAVITY = 0.5  # Gravity strength (acceleration downward)
FRICTION = 0.98  # Friction coefficient (values close to 1 mean less friction)
BOUNCE_DAMPING = 0.8  # Energy loss when bouncing (values close to 1 mean less energy loss)

# Create the screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Bouncing Ball in Spinning Hexagon")
clock = pygame.time.Clock()

# Center of the screen
center_x, center_y = WIDTH // 2, HEIGHT // 2

# Hexagon properties
hex_radius = 200  # Distance from center to vertices
hex_angle = 0  # Current rotation angle (in radians)
hex_rotation_speed = 0.01  # Rotation speed in radians per frame

# Ball properties
ball_radius = 20
ball_pos = [center_x, center_y - 100]  # Start above center
ball_vel = [2, 0]  # Initial velocity [x, y]

def get_hexagon_vertices():
    """Calculate the current hexagon vertices based on rotation angle"""
    vertices = []
    for i in range(6):
        # Calculate angle for each vertex (60 degrees or pi/3 radians apart)
        angle = hex_angle + i * (2 * math.pi / 6)
        # Convert polar coordinates to Cartesian coordinates
        x = center_x + hex_radius * math.cos(angle)
        y = center_y + hex_radius * math.sin(angle)
        vertices.append((x, y))
    return vertices

def get_hexagon_edges():
    """Get the line segments that make up the hexagon"""
    vertices = get_hexagon_vertices()
    edges = []
    for i in range(6):
        # Connect each vertex to the next one
        start = vertices[i]
        end = vertices[(i + 1) % 6]  # % 6 wraps around to the first vertex
        edges.append((start, end))
    return edges

def distance_point_to_line(point, line):
    """
    Calculate the distance from a point to a line segment
    and the closest point on the line
    """
    x, y = point
    (x1, y1), (x2, y2) = line
    
    # Vector from line start to end
    line_vec_x = x2 - x1
    line_vec_y = y2 - y1
    
    # Vector from line start to point
    point_vec_x = x - x1
    point_vec_y = y - y1
    
    # Length of line squared
    line_len_sq = line_vec_x**2 + line_vec_y**2
    
    # Calculate projection ratio (how far along the line the closest point is)
    # t = 0 means closest to start point, t = 1 means closest to end point
    # Clamped between 0 and 1 for points outside the line segment
    if line_len_sq == 0:  # Line has zero length
        t = 0
    else:
        t = max(0, min(1, (point_vec_x * line_vec_x + point_vec_y * line_vec_y) / line_len_sq))
    
    # Calculate closest point on line
    closest_x = x1 + t * line_vec_x
    closest_y = y1 + t * line_vec_y
    closest_point = (closest_x, closest_y)
    
    # Calculate distance
    distance = math.sqrt((x - closest_x)**2 + (y - closest_y)**2)
    
    return distance, closest_point

def main():
    global hex_angle, ball_pos, ball_vel
    
    running = True
    while running:
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                # Press space to reset ball position
                if event.key == pygame.K_SPACE:
                    ball_pos = [center_x, center_y - 100]
                    ball_vel = [2, 0]
        
        # Clear the screen
        screen.fill(BLACK)
        
        # Update hexagon angle for rotation
        hex_angle += hex_rotation_speed
        
        # Draw hexagon
        hexagon_vertices = get_hexagon_vertices()
        pygame.draw.polygon(screen, WHITE, hexagon_vertices, 2)  # 2 is line thickness
        
        # Apply gravity to ball velocity (increase downward speed)
        ball_vel[1] += GRAVITY
        
        # Apply friction to ball velocity (slow down over time)
        ball_vel[0] *= FRICTION
        ball_vel[1] *= FRICTION
        
        # Update ball position based on velocity
        next_pos_x = ball_pos[0] + ball_vel[0]
        next_pos_y = ball_pos[1] + ball_vel[1]
        
        # Check for collisions with hexagon edges
        hexagon_edges = get_hexagon_edges()
        
        for edge in hexagon_edges:
            # Get distance and closest point to the edge
            distance, closest_point = distance_point_to_line([next_pos_x, next_pos_y], edge)
            
            # If collision detected
            if distance < ball_radius:
                # Calculate normal vector from closest point to ball center
                # This vector points away from the wall, which is the direction the ball should bounce
                normal_x = next_pos_x - closest_point[0]
                normal_y = next_pos_y - closest_point[1]
                
                # Normalize the normal vector (make it length 1)
                normal_length = math.sqrt(normal_x**2 + normal_y**2)
                if normal_length > 0:  # Avoid division by zero
                    normal_x /= normal_length
                    normal_y /= normal_length
                
                # Calculate the dot product of velocity and normal
                # This tells us how much the ball is moving towards the wall
                dot_product = ball_vel[0] * normal_x + ball_vel[1] * normal_y
                
                # Only bounce if the ball is moving towards the edge
                if dot_product < 0:
                    # Reflect velocity vector about the normal vector
                    # This is the physics formula for reflection
                    ball_vel[0] -= 2 * dot_product * normal_x
                    ball_vel[1] -= 2 * dot_product * normal_y
                    
                    # Apply bounce damping (energy loss)
                    ball_vel[0] *= BOUNCE_DAMPING
                    ball_vel[1] *= BOUNCE_DAMPING
                    
                    # Move ball outside of collision to prevent getting stuck
                    penetration = ball_radius - distance
                    next_pos_x += normal_x * penetration
                    next_pos_y += normal_y * penetration
        
        # Update ball position
        ball_pos[0] = next_pos_x
        ball_pos[1] = next_pos_y
        
        # Draw ball
        pygame.draw.circle(screen, RED, (int(ball_pos[0]), int(ball_pos[1])), ball_radius)
        
        # Update the display
        pygame.display.flip()
        
        # Cap the frame rate
        clock.tick(FPS)

    # Clean up
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()